In [ ]:
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 150)


import cv2
import numpy as np
from scipy.sparse import spdiags
from scipy.sparse.linalg import spsolve

In [ ]:
img = cv2.imread('/tmp/avaout.png').astype(np.float32)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img,dsize=(500,500))
img = img/img.max()


L = cv2.imread('../data/external/ava.jpg').astype(np.float32)
# L = cv2.cvtColor(L, cv2.COLOR_BGR2GRAY)
L = cv2.resize(L,dsize=(500,500))
L = L/L.max()

In [ ]:
def wlsFilter(IN, lamb=1, alpha=1.2, L=None): 
#     L = np.log(IN+np.spacing(1))
    
    smallNum = 0.0001
    
    r,c = IN.shape
    k = r*c
    
    dy = np.diff(L,1,0)

    dy = -lamb/(np.abs(dy)**alpha + smallNum)
    dy = np.pad(dy, ((0 ,1),(0,0)),mode='constant')
    dy = dy.flatten()
    
    dx = np.diff(L, 1, 1); 
    dx = -lamb/(np.abs(dx)**alpha + smallNum);
    dx = np.pad(dx, ((0 ,0),(0,1)),mode='constant')
    dx = dx.flatten()
    
    print(dx.shape[0])
    print(dy.shape[0])

    B = np.zeros(shape=(dx.shape[0],2))
    B[:,0] = dx
    B[:,1] = dy;
    print(B.shape)
    d = np.array([-r,-1])

    A = spdiags(B.T,d,k,k)
    
    e = dx
    w = np.pad(dx, ((r,0)) ,mode= 'constant')
    w = w[0:-r]
    
    s = dy
    n = np.pad(dy, ((1,0)), mode= 'constant')
    n = n[0:-1]
    
    D = 1-(e+w+s+n)
    A = A + A.T + spdiags(D.T, 0, k, k)
#     print(type(A))
    OUT = spsolve(A,IN.flatten())
#     print(r)
#     print(c)
    OUT = np.reshape(OUT, (r, c))
    
    return OUT
    

In [ ]:
# output = wlsFilter(img,lamb=1.2,alpha=.8,L=L)

In [ ]:
output = np.zeros_like(img)
lamb = .8
alpha = .9

output[:,:,0] = wlsFilter(img[:,:,0],lamb=lamb,alpha=alpha,L=L[:,:,0])
output[:,:,1] = wlsFilter(img[:,:,1],lamb=lamb,alpha=alpha,L=L[:,:,1])
output[:,:,2] = wlsFilter(img[:,:,2],lamb=lamb,alpha=alpha,L=L[:,:,2])

In [ ]:
output2 = np.zeros_like(img)
lamb = 2.2
alpha = 1.2 

output2[:,:,0] = wlsFilter(img[:,:,0],lamb=lamb,alpha=alpha,L=output2[:,:,0])
output2[:,:,1] = wlsFilter(img[:,:,1],lamb=lamb,alpha=alpha,L=output2[:,:,1])
output2[:,:,2] = wlsFilter(img[:,:,2],lamb=lamb,alpha=alpha,L=output2[:,:,2])

In [ ]:
plt.imshow(np.clip(output + img - output2,0,1)[:,:,::-1])

In [ ]:
# output = output.astype(np.int)
plt.imshow(output[:,:,::-1])

In [ ]:
plt.axis('off')
plt.imshow(img[:,:,::-1],cmap='gray')
plt.show()
kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])

im = cv2.filter2D(output, -1, kernel)
im = np.clip(im,0,1)
plt.axis('off')
plt.imshow(im[:,:,::-1],cmap='gray')

